In [6]:
import pandas as pd
import numpy as np
from CounterSpeechGenerator import process_train_data

Setting up an example: list of hate speech, list of "gpt2" counterspeech, list of "gpt3" counterspeech (not actually, just pretending)

In [4]:
hs_cs_pairs, targets = process_train_data()

In [5]:
hatespeech = [hs for (hs, _, _) in hs_cs_pairs]
counterspeech = [cs for (_, cs, _) in hs_cs_pairs]

In [7]:
hatespeech = np.array(hatespeech)

In [8]:
unique, counts = np.unique(hatespeech, return_counts=True)
duplicates = unique[counts > 1]

In [10]:
duplicates = duplicates[:5]

In [22]:
hatespeeches = []
target_labels = []
gpt2_counterspeech = []
gpt3_counterspeech = []

In [23]:
for hs in duplicates:
    indices = np.where(hatespeech == hs)[0]
    cs1 = counterspeech[indices[0]]
    cs2 = counterspeech[indices[1]]
    target_labels.append(targets[indices[0]])
    hatespeeches.append(hs)
    gpt2_counterspeech.append(cs1)
    gpt3_counterspeech.append(cs2)

In [24]:
hatespeeches

['99% of Muslims in the UK are unemployable and refuse to work.',
 "A Muslim's only real allegiance is to Allah (as a comment in relation to election for public office etc.)",
 'According to the Bible, then gays are an abomination to God.',
 'Afghanistan or maybe Pakistan? I bet this does not look like an English town to you? #saville Town (Text accompanying video footage of people on their way to Friday prayers etc. )',
 'Africans are the cause of all the ills in the world. Plus, their bodies are weak and we are born to dominate them.']

In [25]:
target_labels

['MUSLIMS', 'MUSLIMS', 'LGBT+', 'MUSLIMS', 'POC']

In [26]:
gpt2_counterspeech

['That is a blatant lie. (attach unemployment statistics)',
 'So their faith and belief in the teachings of God guides them to a virtuous path of public service. What exactly are you complaining about?',
 'The Bible clearly teaches that everyone is born equal in dignity with the Lord.',
 'Small groups of people peacefully gather to pray, but because they do not look like you, you are outraged? Why am I not surprised?',
 "Aree you suggesting that we shouldn't live with Africans? To think that we are genetically superior to them because of their physical characteristics is just wrong."]

In [27]:
gpt3_counterspeech

['The vast majority of them bring necessary skills into our economy/society. Plus, they are eager to work. I would suggest to check your sources?',
 'So you are saying that we should only allow non-religious people into public office?',
 "Homosexuality has been practiced throughout history in all major civilizations. It is just that today's people accept homosexuality as normal and accept people of other backgrounds too.",
 'Omg brown people contributing to our society. How dare they!?',
 'Well, the argument sounds like you are trying to justify racial hatred by making some generalisations. Is that the case?']

Creating a df comparing the generated responses:

In [29]:
data = {}
for i, hs in enumerate(hatespeeches):
    target = target_labels[i]
    if np.random.random() < 0.5:
        model1_cs = gpt2_counterspeech[i]
        model2_cs = gpt3_counterspeech[i]
        model1_is_gpt2 = True
    else:
        model2_cs = gpt2_counterspeech[i]
        model1_cs = gpt3_counterspeech[i]
        model1_is_gpt2 = False
    data[i] = {
        'Hatespeech': hs,
        'Target': target,
        'Model 1 CS': model1_cs,
        'Model 2 CS': model2_cs,
        'Model 1 is GPT2': model1_is_gpt2
    }

In [30]:
data

{0: {'Hatespeech': '99% of Muslims in the UK are unemployable and refuse to work.',
  'Target': 'MUSLIMS',
  'Model 1 CS': 'The vast majority of them bring necessary skills into our economy/society. Plus, they are eager to work. I would suggest to check your sources?',
  'Model 2 CS': 'That is a blatant lie. (attach unemployment statistics)',
  'Model 1 is GPT2': False},
 1: {'Hatespeech': "A Muslim's only real allegiance is to Allah (as a comment in relation to election for public office etc.)",
  'Target': 'MUSLIMS',
  'Model 1 CS': 'So their faith and belief in the teachings of God guides them to a virtuous path of public service. What exactly are you complaining about?',
  'Model 2 CS': 'So you are saying that we should only allow non-religious people into public office?',
  'Model 1 is GPT2': True},
 2: {'Hatespeech': 'According to the Bible, then gays are an abomination to God.',
  'Target': 'LGBT+',
  'Model 1 CS': "Homosexuality has been practiced throughout history in all majo

In [31]:
df = pd.DataFrame(data)

In [34]:
df = df.T

Write to excel file:

In [35]:
df.to_excel('gpt_comparison.xlsx')